<a href="https://colab.research.google.com/github/Agewerc/wildlifeComputerVision/blob/master/wild_animal_computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Networks

## Project: Write an Algorithm for a Wildlife Identification App 

---



>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut.  Markdown cells can be edited by double-clicking the cell to enter edit mode.

The rubric contains _optional_ "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. If you decide to pursue the "Stand Out Suggestions", you should include the code in this Jupyter notebook.


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = '/content/gdrive/My Drive/oregon_wildlife'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [13]:
import numpy as np
from glob import glob
# Imports
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from skimage import io
from torch.utils.data import (Dataset, DataLoader, )  # Gives easier dataset managment and creates mini batches
from torchvision.datasets import ImageFolder 
import pandas as pd
import os
import re
from sklearn.preprocessing import LabelEncoder
import random 

In [3]:
# load filenames for human and dog images
bald_eagle = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/bald_eagle/*"))
black_bear = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/black_bear/*"))
bob_cat = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/bobcat/*"))
canada_lynx = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/canada_lynx/*"))
columbian_black_tailed_deer = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/columbian_black-tailed_deer/*"))
cougar = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/cougar/*"))
coyote = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/coyote/*"))
deer = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/deer/*"))
elk = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/elk/*"))
gray_fox = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/gray_fox/*"))
gray_wolf = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/gray_wolf/*"))
mountain_beaver = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/mountain_beaver/*"))
nutria = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/nutria/*"))
raccoon = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/raccoon/*"))
raven = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/raven/*"))
red_fox = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/red_fox/*"))
ringtail = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/ringtail/*"))
seals = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/seals/*"))
virginia_opossum = np.array(glob("/content/gdrive/My Drive/oregon_wildlife/virginia_opossum/*"))

In [4]:
# print number of images in each dataset
print('There are %d total bald_eagle images.' % len(bald_eagle))
print('There are %d total black_bear images.' % len(black_bear))
print('There are %d total bob_cat images.' % len(bob_cat))
print('There are %d total canada_linx images.' % len(canada_lynx))
print('There are %d total columbian_black_tailed_deer images.' % len(columbian_black_tailed_deer))
print('There are %d total cougar images.' % len(cougar))
print('There are %d total coyote images.' % len(coyote))
print('There are %d total deer images.' % len(deer))
print('There are %d total elk images.' % len(elk))
print('There are %d total gray_fox images.' % len(gray_fox))
print('There are %d total gray_wolf images.' % len(gray_wolf))
print('There are %d total mountain_beaver images.' % len(mountain_beaver))
print('There are %d total nutria images.' % len(nutria))
print('There are %d total raccoon images.' % len(raccoon))
print('There are %d total raven images.' % len(raven))
print('There are %d total red_fox images.' % len(red_fox))
print('There are %d total ringtail images.' % len(ringtail))
print('There are %d total seals images.' % len(seals))
print('There are %d total virginia_opossum images.' % len(virginia_opossum))


There are 748 total bald_eagle images.
There are 718 total black_bear images.
There are 696 total bob_cat images.
There are 717 total canada_linx images.
There are 735 total columbian_black_tailed_deer images.
There are 686 total cougar images.
There are 736 total coyote images.
There are 764 total deer images.
There are 660 total elk images.
There are 668 total gray_fox images.
There are 730 total gray_wolf images.
There are 577 total mountain_beaver images.
There are 701 total nutria images.
There are 728 total raccoon images.
There are 656 total raven images.
There are 759 total red_fox images.
There are 588 total ringtail images.
There are 698 total seals images.
There are 728 total virginia_opossum images.


In [5]:
import glob

dir = '/content/gdrive/My Drive/oregon_wildlife'

files = [f for f in glob.glob(dir + "**/**", recursive=True)]

In [6]:
df_animals = pd.DataFrame({"file_path":files}) 

In [7]:
df_animals['animal'] = df_animals['file_path'].str.extract('/oregon_wildlife/(.+)/')
df_animals['file'] = df_animals['file_path'].str.extract('oregon_wildlife/.+/(.+)')
df_animals = df_animals.dropna()

In [8]:
lb_make = LabelEncoder()
df_animals["animal"] = lb_make.fit_transform(df_animals["animal"])

In [9]:
animal_set = set(df_animals['animal'])
train_val_test_list = [0,1,2]
train_val_weights = [70,15,15]
df_animals['train_val_test'] = 'NA' 

for an in animal_set:
  n = sum(df_animals['animal'] == an)
  train_val_test = random.choices(train_val_test_list, weights= train_val_weights,  k=n)
  df_animals.loc[df_animals['animal'] == an, 'train_val_test'] = train_val_test 

In [10]:
transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),    
}


In [11]:
def check_train(path):
    return (df_animals[df_animals['file_path'] == path].train_val_test == 0).bool

def check_valid(path):
    return (df_animals[df_animals['file_path'] == path].train_val_test == 1).bool

def check_test(path):
    return (df_animals[df_animals['file_path'] == path].train_val_test == 2).bool

In [15]:
dir = "/content/gdrive/My Drive/oregon_wildlife"

In [16]:
# Reading Dataset
image_datasets = {
    'train' : ImageFolder(root= dir, transform=transform['train'], is_valid_file=check_train),
    'valid' : ImageFolder(root=dir, transform=transform['valid'], is_valid_file=check_valid),
    'test' : ImageFolder(root=dir, transform=transform['test'], is_valid_file=check_test)
}

In [18]:
num_workers = 0
batch_size = 20

loaders_scratch = {
    'train' : DataLoader(image_datasets['train'], shuffle = True, batch_size = batch_size),
    'valid' : DataLoader(image_datasets['valid'], shuffle = True, batch_size = batch_size),
    'test' : DataLoader(image_datasets['test'], shuffle = True, batch_size = batch_size)    
}

In [21]:
for batch_idx, (data, target) in enumerate(loaders_scratch['valid']):
  print(data)

tensor([[[[-0.2513, -0.4054, -0.4054,  ..., -0.4739, -0.4911, -0.4226],
          [-0.5596, -0.4568, -0.4054,  ..., -0.6452, -0.6109, -0.5767],
          [-0.3369, -0.3883, -0.3712,  ..., -0.6623, -0.6623, -0.9877],
          ...,
          [-0.1143, -0.2513, -0.0801,  ..., -0.4226, -0.4911, -0.4054],
          [-0.3027, -0.3027, -0.0801,  ..., -0.2171, -0.4911, -0.5938],
          [-0.2513, -0.1657,  0.0056,  ..., -0.1314, -0.4054, -0.5082]],

         [[-0.0749, -0.2325, -0.2675,  ..., -0.3725, -0.3725, -0.3025],
          [-0.4251, -0.3375, -0.2850,  ..., -0.5476, -0.4951, -0.4601],
          [-0.1975, -0.2500, -0.2325,  ..., -0.5476, -0.5651, -0.8627],
          ...,
          [ 0.0301, -0.1275,  0.0476,  ..., -0.3375, -0.4076, -0.3025],
          [-0.1625, -0.1450,  0.0651,  ..., -0.1275, -0.4076, -0.4951],
          [-0.1275,  0.0126,  0.1877,  ..., -0.0399, -0.3025, -0.3901]],

         [[ 0.2348,  0.0779,  0.0605,  ..., -0.0615, -0.0615,  0.0082],
          [-0.1487, -0.0790, -

KeyboardInterrupt: ignored

In [43]:
import glob

dir = '/content/gdrive/My Drive/oregon_wildlife'

files = [f for f in glob.glob(dir + "**/**", recursive=True)]

In [44]:
df_animals = pd.DataFrame({"file_path":files}) 

In [45]:
df_animals['animal'] = df_animals['file_path'].str.extract('/oregon_wildlife/(.+)/')
df_animals['file'] = df_animals['file_path'].str.extract('oregon_wildlife/.+/(.+)')
df_animals = df_animals.dropna()

In [46]:
lb_make = LabelEncoder()
df_animals["animal"] = lb_make.fit_transform(df_animals["animal"])

In [47]:
count_animals = df_animals.animal.value_counts().to_frame().reset_index()

In [48]:
count_animals['count_train'] = round(count_animals['animal'] * 0.7, 0)  
count_animals['count_validation'] = round(count_animals['animal'] * 0.15, 0)
count_animals['count_test'] =count_animals['animal'] - count_animals['count_train'] - count_animals['count_validation']

In [49]:
animal_set = set(df_animals['animal'])

In [50]:
animal_set = set(df_animals['animal'])
train_val_test_list = [0,1,2]
train_val_weights = [70,15,15]
df_animals['train_val_test'] = 'NA' 

for an in animal_set:
  n = sum(df_animals['animal'] == an)
  train_val_test = random.choices(train_val_test_list, weights= train_val_weights,  k=n)
  df_animals.loc[df_animals['animal'] == an, 'train_val_test'] = train_val_test 

In [51]:
c = df_animals[df_animals['train_val_test'] == 0]
df_animals_val = df_animals[df_animals['train_val_test'] == 1]
df_animals_test = df_animals[df_animals['train_val_test'] == 2]


In [52]:
df_animals_train.to_csv('/content/gdrive/My Drive/oregon_wildlife/animals_train.csv', index=False)
df_animals_val.to_csv('/content/gdrive/My Drive/oregon_wildlife/animals_val.csv', index=False)
df_animals_test.to_csv('/content/gdrive/My Drive/oregon_wildlife/animals_test.csv', index=False)

In [53]:
train_dataset = OregonAnimalsDataset(
    csv_file="/content/gdrive/My Drive/oregon_wildlife/animals_train.csv",
    root_dir="/content/gdrive/My Drive/oregon_wildlife/",
    transform=transform['train'],
)

val_dataset = OregonAnimalsDataset(
    csv_file="/content/gdrive/My Drive/oregon_wildlife/animals_val.csv",
    root_dir="/content/gdrive/My Drive/oregon_wildlife/",
    transform=transform['valid'],
)

test_dataset = OregonAnimalsDataset(
    csv_file="/content/gdrive/My Drive/oregon_wildlife/animals_test.csv",
    root_dir="/content/gdrive/My Drive/oregon_wildlife/",
    transform=transform['test'],
)


In [54]:
transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),    
}


In [55]:
num_workers = 0
batch_size = 20

In [56]:
num_workers = 0
batch_size = 20

loaders_scratch = {
    'train' : DataLoader(train_dataset, shuffle = True, batch_size = batch_size),
    'valid' : DataLoader(val_dataset, shuffle = True, batch_size = batch_size),
    'test' : DataLoader(test_dataset, shuffle = True, batch_size = batch_size)    
}

In [87]:
def check_image(path):
  
  # print(path)
  # return df_animals[df_animals['file_path'] == path].train_val_test
  return (df_animals[df_animals['file_path'] == path].train_val_test == 0).bool
  


In [88]:
train_data = torchvision.datasets.ImageFolder(root="/content/gdrive/My Drive/oregon_wildlife/", transform=transform['train'], is_valid_file=check_image)
val_data = torchvision.datasets.ImageFolder(root="/content/gdrive/My Drive/oregon_wildlife/", transform=transform['train'], is_valid_file=check_image)
test_data = torchvision.datasets.ImageFolder(root="/content/gdrive/My Drive/oregon_wildlife/", transform=transform['train'], is_valid_file=check_image)



In [89]:
train_loader = DataLoader(train_data, shuffle = True, batch_size = batch_size)

In [91]:
for batch_idx, (data, target) in enumerate(train_loader):
  print(target)


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


tensor([15,  6, 15, 14, 15,  0, 12, 18, 15,  7,  2,  7, 16, 12, 12,  1,  8, 14,
        10,  8])
tensor([ 9, 17, 10, 10,  4,  2, 11, 16, 11,  6, 10,  3, 14, 10,  6,  1, 14,  0,
        15,  7])
tensor([18,  9,  0,  0,  2,  3,  8,  7,  8, 15, 17, 17,  8, 11, 15,  9, 14,  7,
        13, 19])
tensor([15, 19,  1,  7, 19, 18,  8,  3,  0,  2, 12,  4,  5,  9,  3,  7, 15,  4,
        11,  4])
tensor([10, 12,  7,  4,  8, 17, 11,  7,  4, 17, 18,  9, 17,  9,  0,  5,  9, 14,
        13,  8])
tensor([ 2,  8, 16,  1, 19,  1,  4,  7, 15,  7,  3,  4,  0,  7, 19,  8, 15,  7,
        11,  8])
tensor([15,  7, 11,  6, 15, 10,  7, 11,  0, 16,  7, 16,  2, 17,  1,  2,  1,  0,
        16,  8])


KeyboardInterrupt: ignored